# Setup

In [1]:
# base
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import seaborn as sns
import pandas as pd
import numpy as np
import datetime

In [2]:
import sys
sys.path.insert(1, '/home/mauricio/code/mcr')
from mcr.util import glimpse, plot_value_counts, plot_value_counts_timeseries, missing_report, plot_missing, plot_unique, plot_duplicates, size
from mcr.spark import roem

In [3]:
from pyspark import SparkContext
# SparkContext.getOrCreate(conf: Optional[pyspark.conf.SparkConf] = None) -> 'SparkContext'
SparkContext.setSystemProperty('spark.executor.memory', '2g')
SparkContext.setSystemProperty('spark.executor.cores', '16')
sc = SparkContext.getOrCreate()
# sc.setLogLevel('DEBUG')
# https://intellipaat.com/community/18452/spark-gives-a-stackoverflowerror-when-training-using-als
sc.setCheckpointDir('checkpoint/')
sc.getConf().getAll()

23/05/08 12:52:27 WARN Utils: Your hostname, rig resolves to a loopback address: 127.0.1.1; using 192.168.0.102 instead (on interface enp6s0)
23/05/08 12:52:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/08 12:52:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/08 12:52:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


[('spark.executor.memory', '2g'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.host', '192.168.0.102'),
 ('spark.app.name', 'pyspark-shell'),
 ('spark.app.id', 'local-1683561148303'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.refl

In [4]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').appName('spark_application').getOrCreate()
print(spark.version)

3.4.0


# Introduction to the Million Songs Dataset

Thierry Bertin-Mahieux, Daniel P.W. Ellis, Brian Whitman, and Paul Lamere. The Million Song Dataset. In Proceedings of the 12th International Society for Music Information Retrieval Conference (SIMIR 2012), 2011.

You're going to be using one file of this dataset called The Echo Nest Taste profile dataset. It contains information on over 1 million users including the number of times they've played nearly 400,000 songs. This is more data than we can use for this course, so we will only be using a portion of.

We'll first examine the data, get summary statistics, and then build and evaluate our model. One thing to note here is that because the use of implicit ratings causes ALS to calculate a level of confidence that a user likes a song based on the number of times they've played it, the matrix will need to include zeros for the songs that each user has not yet listened to. In case your data doesn't already include the zeros, we'll walk through how to do this.

In [5]:
msd = spark.read.csv('msd_ratings_w_zeros_16JUL2018.csv',
                     sep=',',
                     header=True,
                     inferSchema=True)
msd.printSchema()
msd.show(3)
msd.count()

root
 |-- userId: integer (nullable = true)
 |-- songId: integer (nullable = true)
 |-- num_plays: integer (nullable = true)

+------+------+---------+
|userId|songId|num_plays|
+------+------+---------+
|   148|   148|        0|
|   243|   496|        0|
|    31|   471|        0|
+------+------+---------+
only showing top 3 rows



234009

In [6]:
# Why msd has duplicated rows by userId, songId that was not consolidated reducing rows from 234009 fot 78003 
# Why without consolidation the ROEM is improved?
# msd = msd.groupby(['userId', 'songId']).agg({'num_plays': 'sum'}).withColumnRenamed('sum(num_plays)', 'num_plays')

In [7]:
msd.show(3)
msd.count()

+------+------+---------+
|userId|songId|num_plays|
+------+------+---------+
|   148|   148|        0|
|   243|   496|        0|
|    31|   471|        0|
+------+------+---------+
only showing top 3 rows



234009

## Cross join intro

In [8]:
users = msd.select("userId").distinct().orderBy('userId')
users.show(3)
users.count()

+------+
|userId|
+------+
|     0|
|     1|
|     2|
+------+
only showing top 3 rows



321

In [9]:
songs = msd.select("songId").distinct().orderBy('songId')
songs.show(3)
songs.count()

+------+
|songId|
+------+
|     0|
|     1|
|     2|
+------+
only showing top 3 rows



729

## Cross join output

Returns the cartesian product with another DataFrame

In [10]:
cross_join = users.crossJoin(songs)
cross_join.show(3)
cross_join.count()

+------+------+
|userId|songId|
+------+------+
|   148|   148|
|   243|   148|
|    31|   148|
+------+------+
only showing top 3 rows



234009

## Joining back original ratings data

In [11]:
msd = cross_join\
    .join(msd, ["userId", "songId"], "left").orderBy(['userId', 'songId'])
msd.show(3)
msd.count()

+------+------+---------+
|userId|songId|num_plays|
+------+------+---------+
|     0|     0|        2|
|     0|     0|        2|
|     0|     0|        2|
+------+------+---------+
only showing top 3 rows



390015

## Filling in with zero

In [12]:
msd = msd.fillna(0, subset='num_plays').persist()
msd.show(3)

+------+------+---------+
|userId|songId|num_plays|
+------+------+---------+
|     0|     0|        2|
|     0|     0|        2|
|     0|     0|        2|
+------+------+---------+
only showing top 3 rows



## Add zeros function

In [13]:
def add_zeros(df, user_col, item_col):
    # Extracts distinct users
    users = df.select(user_col).distinct()
    # Extracts distinct songs
    songs = df.select(item_col).distinct()
    # Joins users and songs, fills blanks with 0
    cross_join = users.crossJoin(songs) \
    .join(df, [user_col, item_col], "left").fillna(0)
    return cross_join

## Exercises

### MSD summary statistics

In [14]:
# Look at the data
msd.show(3)
# Count the number of distinct userIds
user_count = msd.select("userId").distinct().count()
print("Number of users: ", user_count)
# Count the number of distinct songIds
song_count = msd.select("songId").distinct().count()
print("Number of songs: ", song_count)

+------+------+---------+
|userId|songId|num_plays|
+------+------+---------+
|     0|     0|        2|
|     0|     0|        2|
|     0|     0|        2|
+------+------+---------+
only showing top 3 rows

Number of users:  321


Number of songs:  729


In [15]:
# Min num implicit ratings for a song
print("Minimum implicit ratings for a song: ")
msd.filter(F.col("num_plays") > 0).groupBy("songId").count().select(F.min("count")).show()

# Avg num implicit ratings per songs
print("Average implicit ratings per song: ")
msd.filter(F.col("num_plays") > 0).groupBy("songId").count().select(F.avg("count")).show()

# Min num implicit ratings from a user
print("Minimum implicit ratings from a user: ")
msd.filter(F.col("num_plays") > 0).groupBy("userId").count().select(F.min("count")).show()

# Avg num implicit ratings for users
print("Average implicit ratings per user: ")
msd.filter(F.col("num_plays") > 0).groupBy("userId").count().select(F.avg('count')).show()

Minimum implicit ratings for a song: 
+----------+
|min(count)|
+----------+
|         3|
+----------+

Average implicit ratings per song: 
+------------------+
|        avg(count)|
+------------------+
|35.251063829787235|
+------------------+

Minimum implicit ratings from a user: 
+----------+
|min(count)|
+----------+
|        21|
+----------+

Average implicit ratings per user: 
+-----------------+
|       avg(count)|
+-----------------+
|77.42056074766356|
+-----------------+



> Users have at least 21 implicit ratings with an average of 77 and each song has at least 3 implicit ratings with an average of 35.

### Add zeros

In [16]:
Z = pd.DataFrame(
    {
        "num_purchases": {0: 1, 1: 23, 2: 9, 3: 2, 4: 5, 5: 21, 6: 8, 7: 96},
        "productId": {
            0: 777,
            1: 44,
            2: 227,
            3: 1981,
            4: 2390,
            5: 1662,
            6: 1492,
            7: 1811,
        },
        "userId": {0: 2112, 1: 7, 2: 1132, 3: 686, 4: 42, 5: 13, 6: 2112, 7: 22},
    }
)
Z = spark.createDataFrame(Z)
Z.show()
Z.count()

+-------------+---------+------+
|num_purchases|productId|userId|
+-------------+---------+------+
|            1|      777|  2112|
|           23|       44|     7|
|            9|      227|  1132|
|            2|     1981|   686|
|            5|     2390|    42|
|           21|     1662|    13|
|            8|     1492|  2112|
|           96|     1811|    22|
+-------------+---------+------+



8

In [17]:
# Extract distinct userIds and productIds
users = Z.select("userId").distinct()
products = Z.select("productId").distinct()

# Cross join users and products
cj = users.crossJoin(products)

# Join cj and Z
Z_expanded = cj.join(Z, ["userId", "productId"], "left").fillna(0)

# View Z_expanded
Z_expanded.show()
Z_expanded.count()

+------+---------+-------------+
|userId|productId|num_purchases|
+------+---------+-------------+
|    13|      777|            0|
|    42|      227|            0|
|     7|      227|            0|
|  1132|      777|            0|
|  2112|      777|            1|
|    22|      227|            0|
|    13|       44|            0|
|   686|      227|            0|
|    22|      777|            0|
|    42|      777|            0|
|   686|       44|            0|
|     7|       44|           23|
|  2112|       44|            0|
|   686|      777|            0|
|     7|      777|            0|
|  2112|      227|            0|
|    22|       44|            0|
|  1132|      227|            9|
|    42|       44|            0|
|  1132|       44|            0|
+------+---------+-------------+
only showing top 20 rows



56

# Evaluating implicit ratings models

## Evaluating implicit ratings models
Now that we have an implicit ratings dataset, let's discuss these types of models. The first thing you should know is that implicit ratings models have an additional hyperparameter called alpha. Alpha is an integer value that tells Spark how much each additional song play should add to the model's confidence that a user actually likes a song. Like the other hyperparameters, this will need to be tuned through cross validation. The challenge of these models is the evaluation. With explicit ratings, we used was the RMSE. It made sense in that situation because we could match predictions back to a true measure of user preference. In the case of implicit ratings.

## Why RMSE worked before

Predictions reflect actual ratings, RMSE makes sense here.

## Why RMSE doesn't work now

Different metrics, RMSE doesn't make sense here.

We don’t have a true measure of user preference. We only have the number of times a user listened to a song and a measure of how confident our model is that that they like that song. These aren't the same thing and calculating an RMSE between them doesn't make sense. However, using a test set, we can see if our model is giving high predictions to the songs that users have actually listened to. The logic being that if our model is returning a high prediction for a song that the respective user has actually listened to, then the predictions make sense, especially if they've listened to it more than once. We can measure this using the **Rank Ordering Error Metric (ROEM)** 

## (ROEM) Rank Ordering Error Metric

$$ROEM = \frac{\sum_{u,i} r_{u,i}^t rank_{u,i}}{\sum_{u,i} r_{u,i}^t}$$

>  Unfortunately Spark hasn't implemented an evaluator metric like this, so you'll need to build it manually. An ROEM function will be provided to you in subsequent exercises.

## ROEM bad predictions

In [18]:
bad_predictions = spark.createDataFrame(pd.DataFrame(
    columns=["userId", "songId", "plays", "prediction"],
    data=[
        [111, 22, 3, 0.0001],
        [111, 9, 0, 0.999],
        [111, 321, 0, 0.08],
        [222, 84, 0, 0.000003],
        [222, 821, 2, 0.88],
        [222, 91, 2, 0.73],
        [333, 2112, 0, 0.90],
        [333, 42, 2, 0.80],
        [333, 6, 0, 0.01],
    ],
)).sort(['userId', 'prediction'], ascending=[True, False])
bad_predictions.show()

+------+------+-----+----------+
|userId|songId|plays|prediction|
+------+------+-----+----------+
|   111|     9|    0|     0.999|
|   111|   321|    0|      0.08|
|   111|    22|    3|    1.0E-4|
|   222|   821|    2|      0.88|
|   222|    91|    2|      0.73|
|   222|    84|    0|    3.0E-6|
|   333|  2112|    0|       0.9|
|   333|    42|    2|       0.8|
|   333|     6|    0|      0.01|
+------+------+-----+----------+



### Percent Rank

In [19]:
bad_predictions.createOrReplaceTempView("predictions")
#Calculating rankings of songs predictions by user
spark.sql("SELECT userId, songId, plays, prediction, PERCENT_RANK() OVER (PARTITION BY userId ORDER BY prediction DESC) AS percRank FROM predictions")\
     .createOrReplaceTempView("rankings")
spark.sql('SELECT * FROM rankings').show()

+------+------+-----+----------+--------+
|userId|songId|plays|prediction|percRank|
+------+------+-----+----------+--------+
|   111|     9|    0|     0.999|     0.0|
|   111|   321|    0|      0.08|     0.5|
|   111|    22|    3|    1.0E-4|     1.0|
|   222|   821|    2|      0.88|     0.0|
|   222|    91|    2|      0.73|     0.5|
|   222|    84|    0|    3.0E-6|     1.0|
|   333|  2112|    0|       0.9|     0.0|
|   333|    42|    2|       0.8|     0.5|
|   333|     6|    0|      0.01|     1.0|
+------+------+-----+----------+--------+



### Number of plays X rank

In [20]:
spark.sql('SELECT *, plays*percRank as `np*rank` FROM rankings').createOrReplaceTempView("rankings")
spark.sql('SELECT * FROM rankings').show()

+------+------+-----+----------+--------+-------+
|userId|songId|plays|prediction|percRank|np*rank|
+------+------+-----+----------+--------+-------+
|   111|     9|    0|     0.999|     0.0|    0.0|
|   111|   321|    0|      0.08|     0.5|    0.0|
|   111|    22|    3|    1.0E-4|     1.0|    3.0|
|   222|   821|    2|      0.88|     0.0|    0.0|
|   222|    91|    2|      0.73|     0.5|    1.0|
|   222|    84|    0|    3.0E-6|     1.0|    0.0|
|   333|  2112|    0|       0.9|     0.0|    0.0|
|   333|    42|    2|       0.8|     0.5|    1.0|
|   333|     6|    0|      0.01|     1.0|    0.0|
+------+------+-----+----------+--------+-------+



### ROEM calculation

In [21]:
numerator = spark.sql('SELECT SUM(plays * percRank) FROM rankings').collect()[0][0]
denominator = bad_predictions.groupBy().sum('plays').collect()[0][0]
numerator / denominator

0.5555555555555556

In [22]:
roem(spark, bad_predictions, userCol='userId', ratingCol='plays')

0.5555555555555556

## ROEM good predictions

In [23]:
good_predictions = spark.createDataFrame(pd.DataFrame(
    columns=['userId' ,'songId','plays','prediction'],
    data=[[111, 22, 3, 1.1],
          [111, 77, 0, 0.01],
          [111, 99, 0, 0.008],
          [222, 22, 0, 0.0003],
          [222, 77, 2, 1.5],
          [222, 99, 2, 1.4],
          [333, 22, 0, 0.90],
          [333, 77, 2, 1.6],
          [333, 99, 0, 0.01]]))
good_predictions.show()

+------+------+-----+----------+
|userId|songId|plays|prediction|
+------+------+-----+----------+
|   111|    22|    3|       1.1|
|   111|    77|    0|      0.01|
|   111|    99|    0|     0.008|
|   222|    22|    0|    3.0E-4|
|   222|    77|    2|       1.5|
|   222|    99|    2|       1.4|
|   333|    22|    0|       0.9|
|   333|    77|    2|       1.6|
|   333|    99|    0|      0.01|
+------+------+-----+----------+



### Percent Rank

In [24]:
good_predictions.createOrReplaceTempView("predictions")
#Calculating rankings of songs predictions by user
spark.sql("SELECT userId, songId, plays, prediction, PERCENT_RANK() OVER (PARTITION BY userId ORDER BY prediction DESC) AS percRank FROM predictions")\
     .createOrReplaceTempView("rankings")
spark.sql('SELECT * FROM rankings').show()

+------+------+-----+----------+--------+
|userId|songId|plays|prediction|percRank|
+------+------+-----+----------+--------+
|   111|    22|    3|       1.1|     0.0|
|   111|    77|    0|      0.01|     0.5|
|   111|    99|    0|     0.008|     1.0|
|   222|    77|    2|       1.5|     0.0|
|   222|    99|    2|       1.4|     0.5|
|   222|    22|    0|    3.0E-4|     1.0|
|   333|    77|    2|       1.6|     0.0|
|   333|    22|    0|       0.9|     0.5|
|   333|    99|    0|      0.01|     1.0|
+------+------+-----+----------+--------+



### Number of plays X rank

In [25]:
spark.sql('SELECT *, plays*percRank as `np*rank` FROM rankings').createOrReplaceTempView("rankings")
spark.sql('SELECT * FROM rankings').show()

+------+------+-----+----------+--------+-------+
|userId|songId|plays|prediction|percRank|np*rank|
+------+------+-----+----------+--------+-------+
|   111|    22|    3|       1.1|     0.0|    0.0|
|   111|    77|    0|      0.01|     0.5|    0.0|
|   111|    99|    0|     0.008|     1.0|    0.0|
|   222|    77|    2|       1.5|     0.0|    0.0|
|   222|    99|    2|       1.4|     0.5|    1.0|
|   222|    22|    0|    3.0E-4|     1.0|    0.0|
|   333|    77|    2|       1.6|     0.0|    0.0|
|   333|    22|    0|       0.9|     0.5|    0.0|
|   333|    99|    0|      0.01|     1.0|    0.0|
+------+------+-----+----------+--------+-------+



### ROEM calculation

In [26]:
numerator = spark.sql('SELECT SUM(plays * percRank) FROM rankings').collect()[0][0]
denominator = good_predictions.groupBy().sum('plays').collect()[0][0]
numerator / denominator

0.1111111111111111

In [27]:
roem(spark, good_predictions, userCol='userId', ratingCol='plays')

0.1111111111111111

## Building several ROEM models

In [28]:
from pyspark.ml.recommendation import ALS

(train, test) = msd.randomSplit([.8, .2])

In [29]:
# Empty list to be filled with models
model_list = []
# Complete each of the hyperparameter value lists
ranks = [10, 20, 30, 40]
maxIters = [10, 20, 30, 40]
regParams = [.05, .1, .15]
alphas = [20, 40, 60, 80]
# For loop will automatically create and store ALS models
for r in ranks:
    for mi in maxIters:
        for rp in regParams:
            for a in alphas:
                model_list.append(ALS(userCol= "userId", itemCol= "songId",
                                      ratingCol= "num_plays", rank = r, maxIter = mi, regParam = rp,
                                      alpha = a, coldStartStrategy="drop",nonnegative = True,
                                      implicitPrefs = True))
print(len(model_list))

192


## Error output

In [30]:
ROEMS = []
for i, model in enumerate(model_list):
    # Fits each model to the training data
    print(f'Training model #{i} ', end='')
    trained_model = model.fit(train)
    # Generates test predictions
    predictions = trained_model.transform(test)
    # Evaluates each model's performance
    ROEMS.append(roem(spark, predictions, 'userId', 'num_plays'))
    print(f'ROEM:', ROEMS[-1])

Training model #0 ROEM: 0.08874493046678647
Training model #1 ROEM: 0.09663821944522209
Training model #2 ROEM: 0.10328372095011613
Training model #3 ROEM: 0.10724205888235687
Training model #4 ROEM: 0.08848094911677111
Training model #5 ROEM: 0.0957287518738229
Training model #6 ROEM: 0.1017360736868199
Training model #7 ROEM: 0.10580659792092394
Training model #8 ROEM: 0.08738962956366758
Training model #9 ROEM: 0.09497871384949229
Training model #10 ROEM: 0.1002834323182076
Training model #11 ROEM: 0.10478079704851734
Training model #12 ROEM: 0.0883444365544352
Training model #13 ROEM: 0.09465293298858045
Training model #14 ROEM: 0.1003615999313152
Training model #15 ROEM: 0.1008203990074308
Training model #16 ROEM: 0.0875392856326891
Training model #17 ROEM: 0.09363988542951324
Training model #18 ROEM: 0.09732934410211465
Training model #19 ROEM: 0.09712594484135964
Training model #20 ROEM: 0.08690172940195626
Training model #21 ROEM: 0.0934995323916171
Training model #22 ROEM: 0.0

In [31]:
# Find the index of the smallest ROEM
i = np.argmin(ROEMS)
print("Index of smallest ROEM:", i)

# Find ith element of ROEMS
print("Smallest ROEM: ", ROEMS[i])

Index of smallest ROEM: 152
Smallest ROEM:  0.04733565274244826


## Exercises

### Running a cross-validated implicit ALS model

Now that we have several ALS models, each with a different set of hyperparameter values, we can train them on a training portion of the msd dataset using cross validation, and then run them on a test set of data and evaluate how well each one performs using the ROEM function discussed earlier. Unfortunately, this takes too much time for this exercise, so it has been done separately. But for your reference you can evaluate your model_list using the following loop (we are using the msd dataset in this case).

For purposes of walking you through the steps, the test predictions for 192 models have already been generated, and their ROEM has been calculated. They are found in the ROEMS list provided. Because a list isn't unique to Pyspark, and because numpy works really well with lists, we're going to use numpy here. Follow the instructions below to find the best ROEM and the model that provided it.

In [32]:
# Split the data into training and test sets
(training, test) = msd.randomSplit([0.8, 0.2])

#Building 5 folds within the training set.
train1, train2, train3, train4, train5 = training.randomSplit([0.2, 0.2, 0.2, 0.2, 0.2], seed = 1)
fold1 = train2.union(train3).union(train4).union(train5)
fold2 = train3.union(train4).union(train5).union(train1)
fold3 = train4.union(train5).union(train1).union(train2)
fold4 = train5.union(train1).union(train2).union(train3)
fold5 = train1.union(train2).union(train3).union(train4)

foldlist = [(fold1, train1), (fold2, train2), (fold3, train3), (fold4, train4), (fold5, train5)]

ROEMS = []
# Loops through all models and all folds
for model in model_list:
    for ft_pair in foldlist:
        # Fits model to fold within training data
        fitted_model = model.fit(ft_pair[0])
        # Generates predictions using fitted_model on respective CV test data
        predictions = fitted_model.transform(ft_pair[1])
        # Generates and prints a ROEM metric CV test data
        r = roem(spark, predictions, 'userId', 'num_plays')
        print ("ROEM: ", r)
    # Fits model to all of training data and generates preds for test data
    v_fitted_model = model.fit(training)
    v_predictions = v_fitted_model.transform(test)
    v_ROEM = roem(spark, v_predictions, 'userId', 'num_plays')
    # Adds validation ROEM to ROEM list
    ROEMS.append(v_ROEM)
    print ("Validation ROEM: ", v_ROEM)

ROEM:  0.08617143998700197


ROEM:  0.08837538667608544


ROEM:  0.08583484653140143


ROEM:  0.0877753310739863


ROEM:  0.0886387865044038
Validation ROEM:  0.08678498620088304


...

ROEM:  0.05513256409507222


ROEM:  0.055628900450711845


ROEM:  0.05196429069014226


ROEM:  0.059512920839545104


ROEM:  0.05734069251731454
Validation ROEM:  0.05180534913323652


> Last executed at 2023-05-05 20:02:54 in 1h 24m 53s

In [33]:
# Find the index of the smallest ROEM
i = np.argmin(ROEMS)
print(f'Smallest ROEM #{i}: {ROEMS[i]}')

Smallest ROEM #164: 0.049110232501098576


### Extracting parameters

In [34]:
# Extract the best_model
best_model = model_list[i]

# Extract the Rank
print ("Rank: ", best_model.getRank())

# Extract the MaxIter value
print ("MaxIter: ", best_model.getMaxIter())

# Extract the RegParam value
print ("RegParam: ", best_model.getRegParam())

# Extract the Alpha value
print ("Alpha: ", best_model.getAlpha())

Rank:  40
MaxIter:  20
RegParam:  0.15
Alpha:  20.0


> Great work. Looks like a low rank, a higher maxIter, a low regParam, and a medium-high alpha is keeping the ROEM low. Because some of these values are on the high and low ends of the values we tried, it would be worth adding some additional values to test in our hyperparameter values, and doing this step again. But for right now, you should understand the process.